# Testing the experiment classes


In [1]:
from torch.utils.data.dataset import T_co
from common.experiment import KubemlExperiment, History, TrainOptions, TrainRequest

c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# read the json
with open('./data/5.json', 'r') as f:
    content = f.read()


print(content)

{
  "id": "b71b1d11",
  "task": {
    "model_type": "example",
    "batch_size": 128,
    "epochs": 5,
    "dataset": "mnist",
    "lr": 0.01,
    "function_name": "network",
    "options": {
      "default_parallelism": 5,
      "static_parallelism": true,
      "validate_every": 0,
      "k": 10,
      "goal_accuracy": 90
    }
  },
  "data": {
    "validation_loss": [
      0.2873407344818115
    ],
    "accuracy": [
      91.4
    ],
    "train_loss": [
      3.2260020107030867,
      1.1191881984472274,
      0.6412935853004456,
      0.533321401849389,
      0.43479404859244825
    ],
    "parallelism": [
      5,
      5,
      5,
      5,
      5
    ],
    "epoch_duration": [
      5.311429519,
      2.815807104,
      2.785033174,
      3.161798912,
      2.781011169
    ]
  }
}



In [4]:
# read the content into a class

h = History.from_json(content)

exp = KubemlExperiment("test", None)
exp.history = h
exp.network_id = h.id

exp.save('./tests/')


In [6]:
import pandas as pd
e = pd.read_pickle('./tests/b71b1d11.pkl')
e2 = pd.read_pickle('./tests/other.pkl')

pd.concat([e, e2])

,id,model_type,batch_size,epochs,dataset,lr,function_name,default_parallelism,static_parallelism,validate_every,k,goal_accuracy,validation_loss,accuracy,train_loss,parallelism,epoch_duration
0,b71b1d11,example,128,5,mnist,0.01,network,5,True,0,10,90,[0.2873407344818115],[91.4],"[3.2260020107030867, 1.1191881984472274, 0.641...","[5, 5, 5, 5, 5]","[5.311429519, 2.815807104, 2.785033174, 3.1617..."
0,b71b1d11,example,128,5,mnist,0.01,network,5,True,0,10,90,[0.2873407344818115],[91.4],"[3.2260020107030867, 1.1191881984472274, 0.641...","[5, 5, 5, 5, 5]","[5.311429519, 2.815807104, 2.785033174, 3.1617..."


In [17]:
h.to_dict()

new = {"id": h.id,
       **h.task.to_dict(),
       **h.task.options.to_dict()
       }

del new['options']

new

# # need to wrap the metrics in arrays
for k, v in h.data.to_dict().items():
    new[k] = [v]

new
#
import pandas as pd

pd.DataFrame(new)



,id,model_type,batch_size,epochs,dataset,lr,function_name,default_parallelism,static_parallelism,validate_every,k,goal_accuracy,validation_loss,accuracy,train_loss,parallelism,epoch_duration
0,b71b1d11,example,128,5,mnist,0.01,network,5,True,0,10,90,[0.2873407344818115],[91.4],"[3.2260020107030867, 1.1191881984472274, 0.641...","[5, 5, 5, 5, 5]","[5.311429519, 2.815807104, 2.785033174, 3.1617..."


### Test the experiment class

In [4]:
request = TrainRequest(
    model_type='example',
    function_name='resnet',
    dataset='cifar10',
    lr=0.01,
    batch_size=32,
    epochs= 10,
    options= TrainOptions(
        default_parallelism=5,
        static_parallelism=True,
        validate_every=1,
        k = 10,
        goal_accuracy=90.0
    )
)

exp = KubemlExperiment(title='test experiment', request=request)
exp.network_id = '85878038'
exp.history = h

exp

In [5]:
exp = KubemlExperiment('test', request= request)
exp.network_id = '85878038'

exp.check_if_task_finished()

Checking running tasks C:/Users/diego/CS/thesis/ml/pkg/kubeml-cli/kubeml-cli.exe task list --short
error running command ['C:/Users/diego/CS/thesis/ml/pkg/kubeml-cli/kubeml-cli.exe', 'task', 'list', '--short'] Error: error creating kubernetes client: invalid configuration: no configuration has been provided
Usage:
  kubeml task list [flags]

Flags:
  -h, --help    help for list
      --short   Trigger short format




NameError: name 'exit' is not defined

## Download the datasets

In [6]:
import torch
import torchvision.datasets as datasets

#### Mnist dataset

In [2]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar10_tr = datasets.CIFAR10(root='./datasets/cifar10/', train=True, download=False, transform=None)
cifar10_test = datasets.CIFAR10(root='./datasets/cifar10/', train=False, download=False, transform=None)

In [8]:
import keras
from keras.datasets import mnist, cifar10, cifar100
import numpy as np

file_names = ['x_train', 'y_train', 'x_test', 'y_test']


(15, 32, 32, 3)

In [15]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

# save the different train and test arrays in the datasets folder

for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/mnist/mnist_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)

Saving data of shape (60000, 28, 28), to file ./datasets/mnist/mnist_x_train.npy
Saving data of shape (60000,), to file ./datasets/mnist/mnist_y_train.npy
Saving data of shape (10000, 28, 28), to file ./datasets/mnist/mnist_x_test.npy
Saving data of shape (10000,), to file ./datasets/mnist/mnist_y_test.npy


### Cifar10 dataset

In [11]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()


for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/cifar10/cifar10_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)

Saving data of shape (50000, 32, 32, 3), to file ./datasets/cifar10/cifar10_x_train.npy
Saving data of shape (50000, 1), to file ./datasets/cifar10/cifar10_y_train.npy
Saving data of shape (10000, 32, 32, 3), to file ./datasets/cifar10/cifar10_x_test.npy
Saving data of shape (10000, 1), to file ./datasets/cifar10/cifar10_y_test.npy



### Cifar100 dataset

In [14]:
((trainX, trainY), (testX, testY)) = cifar100.load_data()


for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/cifar100/cifar100_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)

Saving data of shape (50000, 32, 32, 3), to file ./datasets/cifar100/cifar100_x_train.npy
Saving data of shape (50000, 1), to file ./datasets/cifar100/cifar100_y_train.npy
Saving data of shape (10000, 32, 32, 3), to file ./datasets/cifar100/cifar100_x_test.npy
Saving data of shape (10000, 1), to file ./datasets/cifar100/cifar100_y_test.npy


## Try to load the data in a custom made dataset

In [17]:

class Cifar10Dataset(Dataset):

    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def __init__(self):
        # load the train data and labels
        self.data = np.load('datasets/cifar10/cifar10_x_train.npy')
        self.targets = np.load('datasets/cifar10/cifar10_y_train.npy')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.transform(self.data[index]), self.targets[index]

In [18]:
d = Cifar10Dataset()


In [24]:
loader = DataLoader(d, batch_size=16)
i = iter(loader)


In [25]:
a = next(i)

In [27]:
x, y = a
x


tensor([[[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
          [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
          [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
          ...,
          [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
          [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
          [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

         [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
          [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
          [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
          ...,
          [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
          [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
          [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

         [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
          [-0.8431, -1.0000, -

In [1]:
#explore resnet

from torchvision.models.resnet import resnet18

r = resnet18(pretrained=True)

c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\diego/.cache\torch\hub\checkpoints\resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [2]:
r

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
from torchvision.models.vgg import vgg11

v = vgg11()
v

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 